# Microsoft Movie Analysis

_Author: Valentina Valdez_

![picture of motion picture camera](Images/pexels-donald-tong-66134.jpg)

## 1.0 Business Understanding

### 1.1 Company Vision & High Level Goals

Microsoft's decision to venture into the movie industry marks an exciting strategic shift. By telling compelling stories through film, Microsoft can connect with audiences on an emotional level and establish a stronger presence in popular culture, leading to increased brand awareness. 

Other tech companies, such as Apple and Amazon, have succesfully transitioned into the entertainment industry. By leveraging its extensive technological expertise, vast resources, and global reach, Microsoft can produce high-quality movies that will enhance its brand image and increase its cultural influence.

This research seeks to use the available data to gain valuable insights into trends and consumption patterns, enabling the company to create tailored content that resonates with viewers and maximizes box office success. Within this notebook, I will explore what types of films Microsoft should make to maximize profitability and create positive brand exposure by producing critically acclaimed films. 

### 1.2 Film Industry Background

The period that this analysis will focus on starts from the New Hollywood Era. The New Hollywood era, spanning the late 1960s to the 1970s, was marked by auteur-driven filmmaking. High recognizable directors like Francis Ford Coppola, Martin Scorsese, and Steven Spielberg emerged during this time period and are still relevant today. The era produced groundbreaking films like "The Godfather," "Taxi Driver," and "Jaws," which not only achieved critical acclaim but also achieved significant commercial success.

In the 1980s and 1990s, the blockbuster era took hold, with studios increasingly focusing on  films designed to attract large audiences and generate substantial profits. Directors like James Cameron and Steven Spielberg emerged and became synonymous with blockbuster filmmaking. "Titanic", which was directed by James Cameron, became the first billion dollar movie. It also won multiple awards, including an Oscar for Best Picture. 

In the 21st century, franchises, established intellectial properties (IPs), and cinematic universes are prevalent. Recognizable IPs such as Marvel and Star Wars series domanite the box office, attracting massive audiences and generating billions of dollars in revenue. Other IPs, such as Lego, Barbie, and Nintendo, have also plunged into filmmaking and have seen great success. These blockbusters often combine action, visual effects, and interconnected storytelling to create immersive cinematic experiences. However, strong independent voices in non-franchise cinema exist, with the likes of filmmakers like Quentin Tarantino and Wes Anderson. Prestige film awards, such as the Academy Awards, the Golden Globes, and the Cannes Film Festival, are also highly regarded in the industry and in popular culture.  


### 1.3 Research Goals

Given the above goals and industry background, this analysis will focus on the following topics:
- Limit analysis to films released between 1969 and 2019.
- Investigate the types of films that produce high ROIs. 
- Find directors with experience creating either high ROI, high prestige films, or both. 

Let's dive in!

## 2.0 Data Understanding

This analysis uses a variety of trusted data sources. The datasets will be used to narrow down how Microsoft should  invest in its filmaking efforts. The sources are as follows:

- **IMDB:** Launched in 1990 - and owned by Amazon since 1998 - IMDB is one of the most popular and recognizable databases. This database houses a large amount of information such as directors, writers, genres, and release date.  
- **The Numbers:** This database was started in 1997, and is now the largest freely available database of movie business information. The available data contains information about movie titles, production budgets, and gross revenue data. 
- **The Academy Awards**: This data was created by scraping the <a href="https://awardsdatabase.oscars.org/">academy database</a> for a Kaggle competition. The Academy Awards is considered the most prestigious filmaking award in America, and this data will provide valuable insights in identifying individuals capable of making prestige films. This dataset contains information on Academy Award nominees and winners between the years 1927 and 2023. Access the data <a href="https://www.kaggle.com/datasets/unanimad/the-oscar-award">here</a> .  


This analysis is limited by the information in these datasets, and may not fully encompass the full scope of the filmaking industry. However, the data is current enough that this analysis can still provide valuable insight and guide Microsoft on their next steps.


The first step in this analysis is to understand the data and how we can transform it to gleam insights. First, I am importing the necessary libraries to read the data and perform the necessary analysis. I will review the data sources one by one to determine what needs to be done before we can perform analysis. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import datetime

%matplotlib inline

### 2.1 IMDB

Let's examine IMDB's database first, which is the most extensive of the three. I will need information on movies titles, their genres, and the directors per film. 

In [2]:
#Connect to database

conn = sqlite3.connect("Data/im.db")

#Review tables
imdb_tables = pd.read_sql("""SELECT name FROM sqlite_master;""", conn)
imdb_tables

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [3]:
#Query tables 
movie_basics = pd.read_sql("""SELECT * FROM movie_basics LIMIT 10;""", conn)
movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [11]:
movie_akas = pd.read_sql("""
SELECT movie_id
FROM movie_akas
WHERE language = 'en'
;""", conn)
movie_akas.head(20)

,movie_id
0,tt10027708
1,tt10027708
2,tt1014759
3,tt1014759
4,tt10225354
5,tt10225420
6,tt10308634
7,tt1182315
8,tt1233259
9,tt1263750


Based on these initial queries, I will need to connect the additional tables to add the directors per film.

### 2.2 The Numbers

For this dataset, the goal is to calculate the Return on Investment per film. Additionally, films will be categorized by budget so that the budget can be discribed as low, mid, or high. Eventually, I can tie this information to the IMDB table and identify which films and directors have produced high ROIs. 

In [5]:
#Import data
numbers_df = pd.read_csv('Data/tn.movie_budgets.csv.gz')

#Preview table
numbers_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [6]:
#Review data structure
numbers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


The initial assessment is that this data will need to be converted in the following ways:

- The budget and gross data columns need to be converted to integers. 
- The release_date column needs to be coverted to datetime object.
- There are no null values, but I will need to further examine to make sure that the values are valid .

### 2.3 The Academy Awards

This dataset will be critical in providing information on which films and directors have been critically accalimed. 

In [7]:
#Import Data
oscars_df = pd.read_csv('Data/the_oscar_award.csv')

#Preview Data
oscars_df.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [8]:
#Review data structure 

oscars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10765 entries, 0 to 10764
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      10765 non-null  int64 
 1   year_ceremony  10765 non-null  int64 
 2   ceremony       10765 non-null  int64 
 3   category       10765 non-null  object
 4   name           10761 non-null  object
 5   film           10450 non-null  object
 6   winner         10765 non-null  bool  
dtypes: bool(1), int64(3), object(3)
memory usage: 515.2+ KB


Based on this preliminary review, I will need to do the following:
- Make sure that the name column belongs to only people.
- Find out the missing data from films
- Filter by relevant categories - in this analysis, we will only look at films that won were either nominated or won  in recognizable categories like Best Picture, etc. 

## 3.0 Data Preparation

Now that we have a good understanding of our data, the next step is to clean the datasets and engineer some features.

### 3.1 IMDB

#### 3.1.1 Movie Genres

For the IMDB dataset, I will need the film name, release year, director, and genre.

In [12]:
#Query relevant tables and preview data

movies_data = pd.read_sql(
"""
SELECT primary_title AS title, 
       start_year, 
       genres,
       primary_name
  FROM movie_basics
      JOIN directors 
           USING(movie_id)
      JOIN persons
           USING(person_id)
  WHERE start_year BETWEEN 1969 AND 2019
  AND movie_id IN (SELECT movie_id
                   FROM movie_akas
                   WHERE language = 'en')
;""", conn)

movies_data.head()

,title,start_year,genres,primary_name
0,One Day Before the Rainy Season,2019,"Biography,Drama",Mani Kaul
1,Sabse Bada Sukh,2018,"Comedy,Drama",Hrishikesh Mukherjee
2,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",Raoul Ruiz
3,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",Raoul Ruiz
4,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",Valeria Sarmiento


In [13]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43337 entries, 0 to 43336
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         43337 non-null  object
 1   start_year    43337 non-null  int64 
 2   genres        42802 non-null  object
 3   primary_name  43337 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


In [14]:
#Check for duplicates
duplicates = movies_data[movies_data.duplicated()]
print(len(duplicates))
duplicates.head()

22016


,title,start_year,genres,primary_name
3,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",Raoul Ruiz
5,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",Valeria Sarmiento
8,Children of the Green Dragon,2010,Drama,Bence Miklauzic
9,Children of the Green Dragon,2010,Drama,Bence Miklauzic
11,The Tragedy of Man,2011,"Animation,Drama,History",Marcell Jankovics


In [15]:
# Remove Duplicates
movies_data.drop_duplicates(inplace=True)
print(len(movies_data))
movies_data.head()

21321


,title,start_year,genres,primary_name
0,One Day Before the Rainy Season,2019,"Biography,Drama",Mani Kaul
1,Sabse Bada Sukh,2018,"Comedy,Drama",Hrishikesh Mukherjee
2,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",Raoul Ruiz
4,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",Valeria Sarmiento
6,So Much for Justice!,2010,History,Miklós Jancsó


In [16]:
#Check for extraneous values
for col in movies_data.columns:
    print(col, '\n', movies_data[col].value_counts(normalize=True).head(), '\n')

title 
 ABCs of Death 2                      0.001454
Hongmao and Lantu: Phoenix Rising    0.000750
Wonderland                           0.000610
Secrets of the North Sea             0.000610
Bridges of Sarajevo                  0.000610
Name: title, dtype: float64 

start_year 
 2014    0.113222
2015    0.113175
2013    0.112284
2012    0.108531
2016    0.107312
Name: start_year, dtype: float64 

genres 
 Drama            0.218079
Documentary      0.140973
Comedy           0.061009
Comedy,Drama     0.039148
Drama,Romance    0.026242
Name: genres, dtype: float64 

primary_name 
 Jing Wong          0.000844
Takashi Miike      0.000844
Koichi Sakamoto    0.000797
Poj Arnon          0.000797
Herman Yau         0.000797
Name: primary_name, dtype: float64 



In [17]:
#Drop Null values
movies_data.dropna(inplace=True)
len(movies_data)

20997

### 3.2 The Numbers

This dataset is key to findind out ROI numbers.  

In [18]:
#Check for duplicates
duplicates = numbers_df[numbers_df.duplicated()]
print(len(duplicates))

0


In [19]:
#Check for extraneous values
for col in numbers_df.columns:
    print(col, '\n', numbers_df[col].value_counts(normalize=True).head(), '\n\n')

id 
 4     0.010031
53    0.010031
61    0.010031
65    0.010031
69    0.010031
Name: id, dtype: float64 


release_date 
 Dec 31, 2014    0.004151
Dec 31, 2015    0.003978
Dec 31, 2010    0.002594
Dec 31, 2008    0.002421
Dec 31, 2012    0.002248
Name: release_date, dtype: float64 


movie 
 Home              0.000519
King Kong         0.000519
Halloween         0.000519
Fantastic Four    0.000346
Unforgettable     0.000346
Name: movie, dtype: float64 


production_budget 
 $20,000,000    0.039952
$10,000,000    0.036666
$30,000,000    0.030612
$15,000,000    0.029920
$25,000,000    0.029575
Name: production_budget, dtype: float64 


domestic_gross 
 $0             0.094777
$8,000,000     0.001557
$2,000,000     0.001211
$7,000,000     0.001211
$10,000,000    0.001038
Name: domestic_gross, dtype: float64 


worldwide_gross 
 $0             0.063473
$8,000,000     0.001557
$7,000,000     0.001038
$2,000,000     0.001038
$11,000,000    0.000692
Name: worldwide_gross, dtype: float64 




Unfortunately, this dataset contains some movies that do not have reported worldwide gross numbers. These will be dropped, but first, I will convert the columns to their appropiate data type. 

In [20]:
#Converting release_date column to datetime object
numbers_df['release_date'] = pd.to_datetime(numbers_df['release_date'])

#Filter results to movies released after 1969 - start of the New Hollywood Era 
numbers_df = numbers_df.loc[numbers_df['release_date'] >= '1969']

#Create release_year column with datatype integer

#Review results
numbers_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [21]:
#Convert 'production_budget', 'domestic_gross', 'worldwide_gross' to integers

#Remove extra symbols from string
cols = ['production_budget', 'domestic_gross', 'worldwide_gross']

for col in cols:
    numbers_df[col] = numbers_df[col].str.replace('$','').str.replace(',','')
numbers_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,425000000,760507625,2776345279
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


In [22]:
#Convert columns to integers

cols = ['production_budget', 'domestic_gross', 'worldwide_gross']
numbers_df[cols] = numbers_df[cols].apply(pd.to_numeric, axis=1)
numbers_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5632 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5632 non-null   int64         
 1   release_date       5632 non-null   datetime64[ns]
 2   movie              5632 non-null   object        
 3   production_budget  5632 non-null   int64         
 4   domestic_gross     5632 non-null   int64         
 5   worldwide_gross    5632 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 308.0+ KB


In [23]:
#Drop rows where worldwide_gross is 0. 
numbers_df = numbers_df.loc[numbers_df['worldwide_gross'] > 0]
numbers_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,425000000,760507625,2776345279
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...,...
5775,76,2006-05-26,Cavite,7000,70071,71644
5776,77,2004-12-31,The Mongol King,7000,900,900
5778,79,1999-04-02,Following,6000,48482,240495
5779,80,2005-07-13,Return to the Land of Wonders,5000,1338,1338


Now that the data is clean, we can create the 'ROI' (Return on Investment) column with the following formula:

$$
ROI = \frac{Net Gross}{Production Budget} * 100
$$

In [24]:
#Adding ROI Column
numbers_df['ROI'] = (numbers_df['worldwide_gross'] - numbers_df['production_budget']) \
                    / numbers_df['production_budget'] * 100

numbers_df['ROI'] = numbers_df['ROI'].round(decimals=2)

numbers_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,ROI
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,553.26
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,154.67
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,-57.21
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,324.38
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,315.37


In [25]:
#Add new column "Year" as integer
numbers_df['release_year'] = pd.DatetimeIndex(numbers_df['release_date']).year

numbers_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,ROI,release_year
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,553.26,2009
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,154.67,2011
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,-57.21,2019
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,324.38,2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,315.37,2017


In [26]:
#Budget ranking - first, define function

def budget_ranking(budget):
    if budget < 500000:
        return "low"
    elif budget > 50000000:
        return "high"
    else:
        return "mid"

In [27]:
#Create new column and apply function

numbers_df['budget_level'] = numbers_df['production_budget'].apply(budget_ranking)
numbers_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,ROI,release_year,budget_level
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,553.26,2009,high
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,154.67,2011,high
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,-57.21,2019,high
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,324.38,2015,high
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,315.37,2017,high


### 3.3 The Academy Awards

In [28]:
#Filtering for films made between 1969 and 2019

oscars_df = oscars_df.loc[(oscars_df['year_film'] >= 1969) & (oscars_df['year_film'] <= 2019)]
print("Min year: ", oscars_df['year_film'].min())
print("Max year: ", oscars_df['year_film'].max())


Min year:  1969
Max year:  2019


In [29]:
#Check for duplicates
duplicates = oscars_df[oscars_df.duplicated()]
print(len(duplicates))

6


In [30]:
duplicates

,year_film,year_ceremony,ceremony,category,name,film,winner
6220,1983,1984,56,MUSIC (Original Song),Music by Michel Legrand; Lyric by Alan Bergman...,Yentl,False
7068,1991,1992,64,MUSIC (Original Song),Music by Alan Menken; Lyric by Howard Ashman,Beauty and the Beast,False
7395,1994,1995,67,MUSIC (Original Song),Music by Elton John; Lyric by Tim Rice,The Lion King,False
8864,2007,2008,80,MUSIC (Original Song),Music by Alan Menken; Lyric by Stephen Schwartz,Enchanted,False
8865,2007,2008,80,MUSIC (Original Song),Music by Alan Menken; Lyric by Stephen Schwartz,Enchanted,False
9091,2009,2010,82,MUSIC (Original Song),Music and Lyric by Randy Newman,The Princess and the Frog,False


In [31]:
#Check for extraneous values
for col in oscars_df.columns:
    print(col, '\n', oscars_df[col].value_counts(normalize=True).head(), '\n\n')

year_film 
 2019    0.022350
2017    0.022176
2013    0.022001
2016    0.022001
2012    0.022001
Name: year_film, dtype: float64 


year_ceremony 
 2020    0.022350
2018    0.022176
2013    0.022001
2017    0.022001
2014    0.022001
Name: year_ceremony, dtype: float64 


ceremony 
 92    0.022350
90    0.022176
85    0.022001
89    0.022001
86    0.022001
Name: ceremony, dtype: float64 


category 
 BEST PICTURE                  0.052034
DIRECTING                     0.044526
DOCUMENTARY (Feature)         0.044526
ACTOR IN A SUPPORTING ROLE    0.044526
COSTUME DESIGN                0.044526
Name: category, dtype: float64 


name 
 John Williams    0.007508
France           0.004889
Meryl Streep     0.003667
Woody Allen      0.003143
Italy            0.002968
Name: name, dtype: float64 


film 
 La La Land                                           0.002504
Titanic                                              0.002504
The Shape of Water                                   0.002326
The Lord

Some winners seem to be countries. This is an issue of who wins the award. For this analysis, we will only be looking at best actress, best actor, best director, and best picture awards. 

In [32]:
#Filter by award data types
oscars_df['category'].unique()

array(['ACTOR', 'ACTOR IN A SUPPORTING ROLE', 'ACTRESS',
       'ACTRESS IN A SUPPORTING ROLE', 'ART DIRECTION', 'CINEMATOGRAPHY',
       'COSTUME DESIGN', 'DIRECTING', 'DOCUMENTARY (Feature)',
       'DOCUMENTARY (Short Subject)', 'FILM EDITING',
       'FOREIGN LANGUAGE FILM',
       'MUSIC (Original Score--for a motion picture [not a musical])',
       'MUSIC (Score of a Musical Picture--original or adaptation)',
       'MUSIC (Song--Original for the Picture)', 'BEST PICTURE',
       'SHORT SUBJECT (Cartoon)', 'SHORT SUBJECT (Live Action)', 'SOUND',
       'SPECIAL VISUAL EFFECTS',
       'WRITING (Screenplay--based on material from another medium)',
       'WRITING (Story and Screenplay--based on material not previously published or produced)',
       'JEAN HERSHOLT HUMANITARIAN AWARD', 'HONORARY AWARD',
       'MUSIC (Original Score)', 'MUSIC (Original Song Score)',
       'WRITING (Story and Screenplay--based on factual material or material not previously published or produced)',

In [33]:
#Remove non-relevent awards
oscars_df = oscars_df[(oscars_df['category'].str.contains('act|directing|picture', case=False)) & \
                      (~oscars_df['category'].str.contains('writing|music|short', case=False))]
oscars_df['category'].unique()

array(['ACTOR', 'ACTOR IN A SUPPORTING ROLE', 'ACTRESS',
       'ACTRESS IN A SUPPORTING ROLE', 'DIRECTING', 'BEST PICTURE',
       'ACTOR IN A LEADING ROLE', 'ACTRESS IN A LEADING ROLE'],
      dtype=object)

In [34]:
#Recheck table
oscars_df.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
4668,1969,1970,42,ACTOR,Richard Burton,Anne of the Thousand Days,False
4669,1969,1970,42,ACTOR,Dustin Hoffman,Midnight Cowboy,False
4670,1969,1970,42,ACTOR,Peter O'Toole,"Goodbye, Mr. Chips",False
4671,1969,1970,42,ACTOR,Jon Voight,Midnight Cowboy,False
4672,1969,1970,42,ACTOR,John Wayne,True Grit,True


In [35]:
oscars_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1573 entries, 4668 to 10350
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      1573 non-null   int64 
 1   year_ceremony  1573 non-null   int64 
 2   ceremony       1573 non-null   int64 
 3   category       1573 non-null   object
 4   name           1573 non-null   object
 5   film           1573 non-null   object
 6   winner         1573 non-null   bool  
dtypes: bool(1), int64(3), object(3)
memory usage: 87.6+ KB


In [36]:
#Separate best picture awards from film staff awards
picture_awards = oscars_df[(oscars_df['category'].str.contains('picture', case=False)) &
                           (~oscars_df['category'].str.contains('directing', case=False))]

picture_awards.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
4743,1969,1970,42,BEST PICTURE,"Hal B. Wallis, Producer",Anne of the Thousand Days,False
4744,1969,1970,42,BEST PICTURE,"John Foreman, Producer",Butch Cassidy and the Sundance Kid,False
4745,1969,1970,42,BEST PICTURE,"Ernest Lehman, Producer","Hello, Dolly!",False
4746,1969,1970,42,BEST PICTURE,"Jerome Hellman, Producer",Midnight Cowboy,True
4747,1969,1970,42,BEST PICTURE,"Jacques Perrin and Hamed Rachedi, Producers",Z,False


In [37]:
picture_awards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 298 entries, 4743 to 10350
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      298 non-null    int64 
 1   year_ceremony  298 non-null    int64 
 2   ceremony       298 non-null    int64 
 3   category       298 non-null    object
 4   name           298 non-null    object
 5   film           298 non-null    object
 6   winner         298 non-null    bool  
dtypes: bool(1), int64(3), object(3)
memory usage: 16.6+ KB


In [38]:
cols = ['category', 'name']
for col in cols:
    print(col, '\n', picture_awards[col].value_counts(normalize=True).head(10), '\n\n')

category 
 BEST PICTURE    1.0
Name: category, dtype: float64 


name 
 Lawrence Bender, Producer                           0.010067
Irwin Winkler and Robert Chartoff, Producers        0.010067
Steven Spielberg and Kathleen Kennedy, Producers    0.010067
John Boorman, Producer                              0.006711
John Foreman, Producer                              0.006711
Warren Beatty, Producer                             0.006711
Stanley Kubrick, Producer                           0.006711
James L. Brooks, Producer                           0.006711
Jerome Hellman, Producer                            0.006711
Richard D. Zanuck and David Brown, Producers        0.006711
Name: name, dtype: float64 




In [39]:
#Isolate directors
director_awards = oscars_df[(~oscars_df['category'].str.contains('picture', case=False)) &
                           (oscars_df['category'].str.contains('directing', case=False))]
cols = ['category', 'name']
for col in cols:
    print(col, '\n', director_awards[col].value_counts(normalize=True).head(10), '\n\n')

category 
 DIRECTING    1.0
Name: category, dtype: float64 


name 
 Martin Scorsese     0.035294
Woody Allen         0.027451
Steven Spielberg    0.027451
Robert Altman       0.019608
Clint Eastwood      0.015686
Peter Weir          0.015686
Ridley Scott        0.011765
Sidney Lumet        0.011765
Alexander Payne     0.011765
Ang Lee             0.011765
Name: name, dtype: float64 




## 4.0 Exploratory Data Analysis

### 4.1 Exploring Budgets and Profits

First, lets examine what correlation, if any, does profit have with budget and ROI. 

## Recommendations

After this preliminary review, we recommend that Microsoft invests in the following strategies:



However, much research is still to be done. The biggest recommendation is to continue this research and explore the following: 